In [ ]:
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
//import scala.io.Source

object Control {
    def using[A <: { def close(): Unit }, B](resource: A)(f: A => B): B =
        try {
            f(resource)
        } finally {
            resource.close()
        }
}

import Control._

val en : String = "en"
val de : String = "de"
val fr : String = "fr"

val langs : List[String] = List(en, de, fr)

//println(new java.io.File(".").getCanonicalPath)
using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$en-lemmatizer.txt")) { source => {
  for (line <- source.getLines) {
    //println(line)
  }
}}



import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import scala.io.Source
defined object Control
import Control._
en: String = en
de: String = de
fr: String = fr
langs: List[String] = List(en, de, fr)
